In [62]:
import pandas_datareader as pdr
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import warnings

In [63]:
start = datetime.datetime(1998,1,1)
end = datetime.datetime.now()
df = pdr.get_data_yahoo("AAPL", start, end)

In [79]:
reit_list = ['AMT', 'CUBE', 'DEI', 'PLD', 'FR', 'MAA', 'MGP', 'MPW', 'WY']
reit_dict = {}
for name in reit_list:
    reit_dict[name] = pdr.get_data_yahoo(name, start, end)['Adj Close']
    

In [105]:
mortgage_df = pd.read_csv('data/30mortgage_rates.csv', skiprows = 14)
mortgage_df.columns = ['date', 'mortgage_rate']
mortgage_df.set_index('date', inplace = True)
mortgage_df = mortgage_df[::-1]
mortgage_df = mortgage_df.loc[mortgage_df.index >= '1998']